In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

def merge_files_by_description(file1, file2, output_file):

    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    merged_df = pd.merge(df1, df2, on="description", how="left")

    merged_df = merged_df.drop_duplicates(subset="description")

    merged_df.to_csv(output_file, index=False)

file1 = "/content/drive/MyDrive/Colab Notebooks/cleaned_data_combine (1).csv"
file2 = "/content/drive/MyDrive/Colab Notebooks/Final_Categorized_Data_with_Sentiment_Analysis.csv"
output_file = "/content/drive/MyDrive/Colab Notebooks/finaldata.csv"

merge_files_by_description(file1, file2, output_file)


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/finaldata.csv")

df.drop(columns=['follower_count', 'following_count', 'in_reply_to_text','is_verified','republishers','reply_links','reply_hashtags','reply_mentions'], inplace=True)


df.head()


,copy_count,description,media_type,id,in_reply_to_post_id,in_reply_to_username,in_reply_to_user_id,is_quote,lang,like_count,...,url,username,time-date,time-time,shamsi_date,description_links,description_hashtags,description_mentions,دسته بندی نهایی,احساس نسبت به رجیستری
0,0.0,تصویب چنین مصوبهای نشاندهنده نگاه دولت به مشکل...,text,0,0,Unknown,0,1.0,fa,0,...,https://twitter.com/nedakhodayari1/status/1880...,nedakhodayari1,2025-01-19,16:38:59,1403-10-30,NaN,#رجیستری,NaN,سیاسی-اخلاقی مثبت,مثبت
1,0.0,مصوبه جدید دولت درباره رجیستری ایفون رئیس امور...,text,0,0,Unknown,0,0.0,fa,0,...,https://twitter.com/kalagh_news/status/1880965...,kalagh_news,2025-01-19,16:37:33,1403-10-30,NaN,#آیفون-,NaN,سیاسی-تصمیمات دولت,خنثی
2,0.0,مصوبه جدید دولت درباره رجیستری ایفون رئیس امور...,text,0,0,Unknown,0,0.0,fa,0,...,https://twitter.com/ramin_sabounian/status/188...,ramin_sabounian,2025-01-19,16:36:59,1403-10-30,NaN,NaN,NaN,سیاسی-تصمیمات دولت,خنثی
3,0.0,علی احمدنیا رئس امور اطلاعرسانی دولت در حساب ک...,photo,0,0,Unknown,0,0.0,fa,2,...,https://twitter.com/peivast/status/18809636821...,peivast,2025-01-19,16:31:03,1403-10-30,https://t.co/YW00rrS3kJ,NaN,NaN,سیاسی-تصمیمات دولت,خنثی
4,0.0,سرنوشت رجیستری ایفون بالاخره مشخص شد خوش به حا...,text,0,0,Unknown,0,1.0,fa,0,...,https://twitter.com/carpoori/status/1880962459...,carpoori,2025-01-19,16:26:11,1403-10-30,NaN,NaN,NaN,گرانی-رجستری,خنثی


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6134 entries, 0 to 6133
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   copy_count             2593 non-null   float64
 1   description            6133 non-null   object 
 2   media_type             2593 non-null   object 
 3   id                     6134 non-null   int64  
 4   in_reply_to_post_id    6134 non-null   int64  
 5   in_reply_to_username   6134 non-null   object 
 6   in_reply_to_user_id    6134 non-null   int64  
 7   is_quote               6134 non-null   float64
 8   lang                   2593 non-null   object 
 9   like_count             6134 non-null   int64  
 10  mentioned_ids          6134 non-null   int64  
 11  quoted_id              6134 non-null   int64  
 12  relation.id            6134 non-null   int64  
 13  title                  6134 non-null   object 
 14  url                    6134 non-null   object 
 15  user

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd
import requests
from collections import Counter

stopwords_url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-fa/master/stopwords-fa.txt"
response = requests.get(stopwords_url)
stopwords = response.text.splitlines()

def remove_stopwords(text, stopwords):
    words = text.split()
    return " ".join([word for word in words if word not in stopwords])

def normalize_word(word):
    word = word.lower()
    word = word.replace("ریجستری", "رجیستری")
    word = word.replace("ریجستر", "رجیستری")
    return word

def get_top_20_words(df, stopwords):
    all_words = []

    for text in df['description'].dropna():
        cleaned_text = remove_stopwords(str(text), stopwords)
        cleaned_words = cleaned_text.split()

        all_words.extend([normalize_word(word) for word in cleaned_words])

    word_counts = Counter(all_words)
    return word_counts.most_common(20)

top_20_words = get_top_20_words(df, stopwords)

top_20_words_df = pd.DataFrame(top_20_words, columns=["Word", "Frequency"])

top_20_words_df.to_csv("top_20_words.csv", index=False)

print(top_20_words_df)

        Word  Frequency
0    رجیستری       5437
1      ایفون       4842
2     واردات       1370
3       دولت       1355
4       گوشی       1094
5       مشکل        815
6         حل        773
7       میشه        674
8       درصد        633
9       ازاد        628
10      دلار        618
11     تعرفه        614
12       رفع        582
13      قیمت        581
14  فیلترینگ        516
15       پول        513
16      دیگه        495
17        ۹۶        450
18      داره        439
19       برق        406


In [ ]:
import pandas as pd
from collections import Counter

def remove_stopwords(text, stopwords):
    words = text.split()
    return " ".join([word for word in words if word not in stopwords])

def get_top_word_for_user(df, stopwords):
    user_data = []

    for username, user_group in df.groupby('username'):
        all_words = []

        # Clean text and remove stopwords
        for text in user_group['description'].dropna():
            cleaned_text = remove_stopwords(str(text), stopwords)
            all_words.extend(cleaned_text.split())

        word_counts = Counter(all_words)

        if word_counts:
            most_common_word, frequency = word_counts.most_common(1)[0]
        else:
            most_common_word, frequency = None, 0

        category_intrest = user_group['دسته بندی نهایی'].mode().iloc[0] if not user_group['دسته بندی نهایی'].mode().empty else None
        feeling = user_group['احساس نسبت به رجیستری'].mode().iloc[0] if not user_group['احساس نسبت به رجیستری'].mode().empty else None

        user_data.append({
            'username': username,
            'most_common_word': most_common_word,
            'word_frequency': frequency,
            'category_intrest': category_intrest,
            'feeling': feeling
        })

    return pd.DataFrame(user_data)


user_top_words_df = get_top_word_for_user(df, stopwords)

print(user_top_words_df)

user_top_words_df.to_csv("user_top_words_and_sentiment.csv", index=False)

             username most_common_word  word_frequency     category_intrest  \
0     0011as_bani1366          ممنوعیت               2  کوچیک شمردن رجیستری   
1        0066nikbakht         کمترینها               1  کوچیک شمردن رجیستری   
2             021koka           درمورد               2  کوچیک شمردن رجیستری   
3     0808siavas96535          پزشکیان               2        گرانی-اقتصادی   
4     0_ordibehesht_0          اصلاحات               2  کوچیک شمردن رجیستری   
...               ...              ...             ...                  ...   
4565          zxswerg            ایفون               1  کوچیک شمردن رجیستری   
4566        zynb_prsn              نشد               2  کوچیک شمردن رجیستری   
4567       zynb_z0592           ریجستر               1  کوچیک شمردن رجیستری   
4568        zynbiiiii            دوقشر               1  کوچیک شمردن رجیستری   
4569    zzeynabi38762          ریجستری               5  کوچیک شمردن رجیستری   

     feeling  
0       خنثی  
1       خنثی  
2     

In [ ]:
import pandas as pd

months_dict = {
    "01": "فروردین", "02": "اردیبهشت", "03": "خرداد",
    "04": "تیر", "05": "مرداد", "06": "شهریور",
    "07": "مهر", "08": "آبان", "09": "آذر",
    "10": "دی", "11": "بهمن", "12": "اسفند"
}

def get_persian_month(date):
    month = date.split('-')[1]
    persian_month = months_dict.get(month)
    return persian_month, month

new_data = []

for idx, row in df.iterrows():
    if pd.isna(row['description_mentions']) or pd.isna(row['username']):
        continue

    mentions = []
    if pd.notna(row['description_mentions']):
        mentions.extend(str(row['description_mentions']).split('@'))

    mentions = [mention.strip() for mention in mentions if mention.strip() != ""]

    date = row['time-date']
    persian_month, month_number = get_persian_month(row['shamsi_date'])
    username = row['username']

    for mention in mentions:
        new_data.append({
            'date': date,
            'mentioned_id': mention,
            'username': username,
            'persian_month': persian_month,
            'persian_month_number': month_number
        })

new_df = pd.DataFrame(new_data)

print(new_df.head())

new_df.to_csv("extracted_mentions_with_username_and_month.csv", index=False)



         date   mentioned_id         username persian_month  \
0  2025-01-17         Sae3ds       amirnaji85            دی   
1  2025-01-15         Sae3ds         nima7red            دی   
2  2025-01-15  HashemiSattar  citnanewsagency            دی   
3  2025-01-13   abdolah_abdi  enayat_e0508652            دی   
4  2025-01-12        Hatefna     hajkazebhifa            دی   

  persian_month_number  
0                   10  
1                   10  
2                   10  
3                   10  
4                   10  


In [ ]:
def get_persian_month(date):
    month = date.split('-')[1]
    persian_month = months_dict.get(month)
    return persian_month, month

hashtag_data = []


for idx, row in df.iterrows():
    if pd.notna(row['description_hashtags']):
        hashtags = []
        if pd.notna(row['description_hashtags']):
            hashtags.extend(str(row['description_hashtags']).split())


        date = row['time-date']
        persian_month, month_number = get_persian_month(row['shamsi_date'])
        username = row['username']

        for hashtag in hashtags:
            hashtag_data.append({
                'date': date,
                'hashtag': hashtag,
                'username': username,
                'persian_month': persian_month,
                'persian_month_number': month_number
            })

hashtag_df = pd.DataFrame(hashtag_data)

print(hashtag_df.head())

hashtag_df.to_csv("hashtags_data.csv", index=False)

         date         hashtag         username persian_month  \
0  2025-01-19        #رجیستری   nedakhodayari1            دی   
1  2025-01-19         #آیفون-      kalagh_news            دی   
2  2025-01-19  #رجیستری_آیفون  citnanewsagency            دی   
3  2025-01-19          #سیتنا  citnanewsagency            دی   
4  2025-01-19  #رجیستری_آیفون    tasnimnews_fa            دی   

  persian_month_number  
0                   10  
1                   10  
2                   10  
3                   10  
4                   10  


In [ ]:
link_data = []


for idx, row in df.iterrows():
    if pd.notna(row['description_links']):
        links = []
        if pd.notna(row['description_links']):
            links.extend(str(row['description_links']).split())


        date = row['time-date']
        persian_month, month_number = get_persian_month(row['shamsi_date'])
        username = row['username']

        for link in links:
            link_data.append({
                'date': date,
                'link': link,
                'username': username,
                'persian_month': persian_month,
                'persian_month_number': month_number
            })

link_df = pd.DataFrame(link_data)

print(link_df.head())

link_df.to_csv("links_data.csv", index=False)


         date                     link         username persian_month  \
0  2025-01-19  https://t.co/YW00rrS3kJ          peivast            دی   
1  2025-01-19  https://t.co/cbVSq1qUcw  citnanewsagency            دی   
2  2025-01-19  https://t.co/LOVxpswiKP  citnanewsagency            دی   
3  2025-01-19  https://t.co/OA3WbwN5QH    tasnimnews_fa            دی   
4  2025-01-15  https://t.co/zR8H5xS2wO       sesmaeili8            دی   

  persian_month_number  
0                   10  
1                   10  
2                   10  
3                   10  
4                   10  


In [ ]:
import pandas as pd

dominant_category = df.groupby('username')['دسته بندی نهایی'].agg(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown')
dominant_feeling = df.groupby('username')['احساس نسبت به رجیستری'].agg(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown')

new_df['dominant_category'] = new_df['username'].map(dominant_category)
new_df['dominant_feeling'] = new_df['username'].map(dominant_feeling)

hashtag_df['dominant_category'] = hashtag_df['username'].map(dominant_category)
hashtag_df['dominant_feeling'] = hashtag_df['username'].map(dominant_feeling)

link_df['dominant_category'] = link_df['username'].map(dominant_category)
link_df['dominant_feeling'] = link_df['username'].map(dominant_feeling)


print(new_df.head())

new_df.to_csv("mention_with_category_and_feeling.csv", index=False)
hashtag_df.to_csv("hashtag_with_category_and_feeling.csv", index=False)
link_df.to_csv("link_with_category_and_feeling.csv", index=False)



         date   mentioned_id         username persian_month  \
0  2025-01-17         Sae3ds       amirnaji85            دی   
1  2025-01-15         Sae3ds         nima7red            دی   
2  2025-01-15  HashemiSattar  citnanewsagency            دی   
3  2025-01-13   abdolah_abdi  enayat_e0508652            دی   
4  2025-01-12        Hatefna     hajkazebhifa            دی   

  persian_month_number      dominant_category dominant_feeling  
0                   10          گرانی-رجیستری             منفی  
1                   10    کوچیک شمردن رجیستری             خنثی  
2                   10     سیاسی-تصمیمات دولت             خنثی  
3                   10  مشکل فنی-تعلل رجیستری             مثبت  
4                   10    کوچیک شمردن رجیستری             خنثی  


In [6]:
import pandas as pd

data1 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/content_1403-11-30_16-42_1739884350.xlsx")
data2 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/4_5886388292760378165.xlsx")

merged_data = pd.merge(data1, data2, on=list(set(data1.columns) & set(data2.columns)), how='outer')


filtered_data = merged_data[merged_data["username"].notna() & merged_data["in_reply_to_username"].notna()]

filtered_data.to_csv("for_graph.csv", index=False)

In [ ]:
pip install hazm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 17.2 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9298 sha256=8e27116b960b66c2272b3cd258c3657ae9c6b788ce43eb29e8b567dc74569c32
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [7]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

import importlib
import prepration_twitter
importlib.reload(prepration_twitter)


<module 'prepration_twitter' from '/content/drive/MyDrive/Colab Notebooks/prepration_twitter.py'>

In [8]:
import pandas as pd

data = pd.read_csv("/content/for_graph.csv")

processed_data = prepration_twitter.preprocess_data(data)

print(processed_data.head())

processed_data.to_csv("/content/processed_for_graph.csv", index=False)

                                         description  follower_count  \
0                                                                NaN   
1                                                                NaN   
2  هنوز معلوم نیست این اون دلار سود بازرگانیه که ...             NaN   
3  هنوز معلوم نیست این اون دلار سود بازرگانیه که ...             NaN   
4                                                                NaN   

   following_count  like_count  \
0              NaN           0   
1              NaN           0   
2              NaN           0   
3              NaN           0   
4              NaN           0   

                                                 url       username  \
0  https://twitter.com/09011346854mm/status/18516...  09011346854mm   
1  https://twitter.com/0xbahari/status/1883484058...       0xbahari   
2  https://twitter.com/0xhawre/status/18517168974...        0xhawre   
3  https://twitter.com/0xhawre/status/18517168974...        0xhawre   
4  http

In [9]:
import pandas as pd

def merge_files_by_description(file1, file2, output_file):

    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    merged_df = pd.merge(df1, df2, on="username", how="left")

    merged_df = merged_df.drop_duplicates(subset="username")

    merged_df.to_csv(output_file, index=False)

file1 = "/content/processed_for_graph.csv"
file2 = "/content/drive/MyDrive/Colab Notebooks/finaldata.csv"
output_file = "/content/drive/MyDrive/Colab Notebooks/for_visualize_graph.csv"

merge_files_by_description(file1, file2, output_file)

In [16]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/for_visualize_graph.csv")

In [18]:
selected_data = data[["username", "in_reply_to_username"]].dropna()
selected_data = selected_data.rename(columns={"username": "source", "in_reply_to_username": "target"})
selected_data["Type"] = "Directed"
selected_data.to_csv("edges.csv", index=False)

In [20]:
selected_data = data[["username","احساس نسبت به رجیستری","دسته بندی نهایی"]].dropna()
selected_data = selected_data.rename(columns={"username": "Id","احساس نسبت به رجیستری":"emotion_regostry","دسته بندی نهایی":"categorized"})
selected_data["Label"] = selected_data["Id"]
selected_data = selected_data.drop_duplicates(subset=["Id"])
selected_data.to_csv("nodes.csv", index=False)

In [21]:
selected_data = data[["username", "in_reply_to_username", "time-date"]].dropna()
selected_data = selected_data.rename(columns={"username": "source", "in_reply_to_username": "target", "time-date": "timestamp"})
selected_data["Type"] = "Directed"
selected_data.to_csv("edges_time.csv", index=False)